In [1]:
import torch
import matplotlib.pyplot as plt

# run on 1 L4OS gpu or 4 L4 gpus to have enough RAM save your model and open another studio or restart kernel in a new notebook to test on 
# the holdout set otherwise the kernel will die and you will need to restart everything
# holdout_images = torch.load("/teamspace/studios/written-violet-bdv7/holdout2_images.pt")
# holdout_labels = torch.load("/teamspace/studios/written-violet-bdv7/holdout2_labels.pt")
# SAVE YOUR MODEL FOR TESTING ON HOLDOUT DATASET LATER

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

X_train, X_test, y_train, y_test = train_test_split(
    train_val_images, train_val_labels, test_size=0.2, random_state=42
)

# Move data to GPU
X_train, X_test = X_train.to(device), X_test.to(device)
y_train, y_test = y_train.to(device), y_test.to(device)

# Create DataLoaders for training and testing
batch_size = 64
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Define the model
def get_model(num_classes=2):
    model = models.resnet18(pretrained=True)
    
    # Freeze all layers except the final layer
    for param in model.parameters():
        param.requires_grad = False

    # Replace the final fully connected layer
    model.fc = nn.Sequential(
        nn.Linear(512, 256),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(256, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(128, num_classes)
    )

    return model.to(device)

# Initialize the model, loss function, and optimizer
model = get_model(num_classes=2)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001, weight_decay=0.001)

# Training and validation loop
epochs = 10
for epoch in range(epochs):
    # Training phase
    model.train()
    total_train_loss = 0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = loss_fn(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    # Validation phase
    model.eval()
    total_val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            val_loss = loss_fn(outputs, y_batch)
            total_val_loss += val_loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

    # Calculate metrics
    avg_train_loss = total_train_loss / len(train_loader)
    avg_val_loss = total_val_loss / len(test_loader)
    accuracy = correct / total

    print(f"Epoch [{epoch+1}/{epochs}]")
    print(f"Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Accuracy: {accuracy * 100:.2f}%")


Using device: cuda


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10]
Train Loss: 0.6319, Val Loss: 0.5791, Accuracy: 69.38%
Epoch [2/10]
Train Loss: 0.5808, Val Loss: 0.5513, Accuracy: 71.62%
Epoch [3/10]
Train Loss: 0.5562, Val Loss: 0.5464, Accuracy: 72.12%
Epoch [4/10]
Train Loss: 0.5386, Val Loss: 0.5362, Accuracy: 72.38%
Epoch [5/10]
Train Loss: 0.5209, Val Loss: 0.5206, Accuracy: 73.74%
Epoch [6/10]
Train Loss: 0.5064, Val Loss: 0.5341, Accuracy: 73.24%
Epoch [7/10]
Train Loss: 0.4969, Val Loss: 0.5464, Accuracy: 71.48%
Epoch [8/10]
Train Loss: 0.4827, Val Loss: 0.5608, Accuracy: 71.13%
Epoch [9/10]
Train Loss: 0.4722, Val Loss: 0.4994, Accuracy: 75.05%
Epoch [10/10]
Train Loss: 0.4639, Val Loss: 0.4987, Accuracy: 75.14%


In [1]:
from sklearn.metrics import average_precision_score
import torch.nn.functional as F

# Validation phase after training
model.eval()  # Set model to evaluation mode
all_labels = []
all_probs = []

print("Calculating Average Precision (AP) on test set...")
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        # Get model outputs and probabilities
        outputs = model(X_batch)
        probs = F.softmax(outputs, dim=1)[:, 0]  # Probabilities for the positive class

        # Collect predictions and true labels
        all_probs.extend(probs.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())

# Calculate Average Precision
avg_precision = average_precision_score(all_labels, all_probs)
print(f"Average Precision (AP): {avg_precision:.4f}")


NameError: name 'model' is not defined

In [ ]:
!unzip /teamspace/uploads/FakeAVCeleb_v1.2.zip -d /teamspace/studios/this_studio


In [3]:
!pwd

/teamspace/studios/this_studio


In [1]:
import pandas as pd 

df = pd.read_csv("FakeAVCeleb_v1.2/meta_data2.csv")
df.head()

,source,target1,target2,method,category,type,race,gender,path,Unnamed: 9
0,id00076,-,-,real,A,RealVideo-RealAudio,African,men,00109.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...
1,id00166,-,-,real,A,RealVideo-RealAudio,African,men,00010.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...
2,id00173,-,-,real,A,RealVideo-RealAudio,African,men,00118.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...
3,id00366,-,-,real,A,RealVideo-RealAudio,African,men,00118.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...
4,id00391,-,-,real,A,RealVideo-RealAudio,African,men,00052.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...


In [2]:
df.columns

Index(['source', 'target1', 'target2', 'method', 'category', 'type', 'race',
       'gender', 'path', 'Unnamed: 9'],
      dtype='object')

In [3]:
df["Unnamed: 9"][0]

'FakeAVCeleb/RealVideo-RealAudio/African/men/id00076'

In [4]:
df["method"].value_counts()

method
wav2lip     5015
fsgan       3964
faceswap     730
real         500
Name: count, dtype: int64

In [7]:
filtered_df = df[df['method'].isin(['faceswap', 'real'])]
filtered_df["method"].value_counts()
filtered_df.to_csv("filtered_data.csv", index=False)

In [27]:
import cv2
from mtcnn import MTCNN
import os
import pandas as pd
import random

def extract_random_frame_from_real_videos(csv_path, output_dir, img_size=(256, 256)):
    # Load the dataset
    df = pd.read_csv(csv_path)
    
    real_videos_df = df[df['method'] == 'faceswap']
    
    real_videos_df['full_path'] = real_videos_df.apply(lambda row: os.path.join(row['Unnamed: 9'], row['path']), axis=1)
    video_paths = real_videos_df['full_path']
    
    detector = MTCNN()
    
    os.makedirs(output_dir, exist_ok=True)
    
    for video_path in video_paths:
        if not os.path.exists(video_path):
            print(f"Video file not found: {video_path}")
            continue
        # Read the video
        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        if total_frames > 0:
            random_frame_index = random.randint(0, total_frames - 1)
            cap.set(cv2.CAP_PROP_POS_FRAMES, random_frame_index)
            ret, frame = cap.read()
            if not ret:
                print(f"Failed to read frame {random_frame_index} from {video_path}.")
                cap.release()
                continue

            results = detector.detect_faces(frame)
            if results:
                x, y, width, height = results[0]['box']
                x, y = max(0, x), max(0, y)  

                cropped_face = frame[y:y+height, x:x+width]
                resized_face = cv2.resize(cropped_face, img_size)

                # Save the cropped face
                video_id = os.path.splitext(os.path.basename(video_path))[0]
                face_filename = os.path.join(output_dir, f"{video_id}_random_face.jpg")
                cv2.imwrite(face_filename, resized_face)
                print(f"Saved cropped face to {face_filename}")
            else:
                print(f"No face detected in random frame of {video_path}.")
        else:
            print(f"No frames in video: {video_path}")
        
        cap.release()

# Example usage
csv_path = 'filtered_data.csv'
output_dir = 'output_frames_fake'
extract_random_frame_from_real_videos(csv_path, output_dir)

/tmp/ipykernel_1228/3740506590.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_videos_df['full_path'] = real_videos_df.apply(lambda row: os.path.join(row['Unnamed: 9'], row['path']), axis=1)


Saved cropped face to output_frames_fake/00109_1_random_face.jpg
Saved cropped face to output_frames_fake/00109_10_random_face.jpg
No face detected in random frame of FakeAVCeleb/FakeVideo-RealAudio/African/men/id00076/00109_11.mp4.
Saved cropped face to output_frames_fake/00109_12_random_face.jpg
Saved cropped face to output_frames_fake/00109_2_random_face.jpg
Saved cropped face to output_frames_fake/00109_3_random_face.jpg
No face detected in random frame of FakeAVCeleb/FakeVideo-RealAudio/African/men/id00076/00109_4.mp4.
Saved cropped face to output_frames_fake/00109_6_random_face.jpg
Saved cropped face to output_frames_fake/00109_7_random_face.jpg
No face detected in random frame of FakeAVCeleb/FakeVideo-RealAudio/African/men/id00076/00109_8.mp4.
Saved cropped face to output_frames_fake/00109_9_random_face.jpg
Saved cropped face to output_frames_fake/00010_id01637_5VjcPZm8knM_faceswap_random_face.jpg
Saved cropped face to output_frames_fake/00118_id00476_UgdYVJ6xPYg_faceswap_random

2024-12-06 21:37:18.082899: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [0,48,48,3] vs. [1,1,1,32]


Saved cropped face to output_frames_fake/00019_1_random_face.jpg
Saved cropped face to output_frames_fake/00019_2_random_face.jpg
Saved cropped face to output_frames_fake/00019_3_random_face.jpg
Saved cropped face to output_frames_fake/00094_0_random_face.jpg
Saved cropped face to output_frames_fake/00191_id02005_7_Egh9mW5y4_faceswap_random_face.jpg
Saved cropped face to output_frames_fake/00050_0_random_face.jpg
Saved cropped face to output_frames_fake/00050_id00475_xQjvXRcnPvw_faceswap_random_face.jpg
Saved cropped face to output_frames_fake/00007_0_random_face.jpg
Saved cropped face to output_frames_fake/00027_random_face.jpg
Saved cropped face to output_frames_fake/00053_id01838_q_lUk55OrL0_faceswap_random_face.jpg
Saved cropped face to output_frames_fake/00005_random_face.jpg
Saved cropped face to output_frames_fake/00384_id05252_CMxIX3absYM_faceswap_random_face.jpg
Saved cropped face to output_frames_fake/00010_random_face.jpg
Saved cropped face to output_frames_fake/00017_random

In [1]:
import os
import cv2
import torch
import time
from concurrent.futures import ThreadPoolExecutor
from torchvision import transforms
from itertools import islice

fake_dir = "output_frames_fake"
real_dir = "output_frames_real"

data_transform = transforms.Compose([
    transforms.ToTensor()  
])

def process_image(file_path, label):
    """Load and transform an image."""
    if not os.path.exists(file_path):
        print(f"Image not found: {file_path}")
        return None 

    img = cv2.imread(file_path)
    if img is None:
        print(f"Failed to load image: {file_path}")
        return None 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
    transformed_img = data_transform(img) 
    return transformed_img, label

def chunks(iterable, size):
    """Yield successive n-sized chunks from an iterable."""
    iterator = iter(iterable)
    for first in iterator:
        yield [first, *islice(iterator, size - 1)]

def process_batch(batch):
    """Process a batch of files."""
    results = []
    for file_path, label in batch:
        result = process_image(file_path, label)
        if result is not None:
            results.append(result)
    return results

def process_dataset(image_dir, label, batch_size=250, dataset_name="", num_workers=32):
    """Process dataset in batches using ThreadPoolExecutor and return tensors."""
    image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    rows = [(file_path, label) for file_path in image_files]
    images = []
    labels = []

    start_time = time.time()
    print(f"Starting processing for {dataset_name} dataset...")

    batches = list(chunks(rows, batch_size))  # Split rows into batches
    print(f"{len(batches)} batches to process for {dataset_name}.")
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        for i, results in enumerate(executor.map(process_batch, batches), start=1):
            for res in results:
                images.append(res[0])
                labels.append(res[1])
            print(f"Processed batch {i}/{len(batches)} for {dataset_name}...")

    # Convert to tensors
    images = torch.stack(images)
    labels = torch.tensor(labels, dtype=torch.long)

    end_time = time.time()
    print(f"Finished processing {dataset_name} dataset in {end_time - start_time:.2f} seconds.")
    return images, labels

# Process fake and real datasets
fake_images, fake_labels = process_dataset(fake_dir, label=1, batch_size=20, dataset_name="Fake", num_workers=64)
real_images, real_labels = process_dataset(real_dir, label=0, batch_size=20, dataset_name="Real", num_workers=64)

# Combine datasets
all_images = torch.cat([fake_images, real_images])
all_labels = torch.cat([fake_labels, real_labels])

# Save tensors as separate files
torch.save(all_images, "all_images.pt")
torch.save(all_labels, "all_labels.pt")

print("Datasets saved as separate files:")
print("All Images: 'all_images.pt', All Labels: 'all_labels.pt'")
print(f"All Images Shape: {all_images.shape}, All Labels Shape: {all_labels.shape}")

Starting processing for Fake dataset...
10 batches to process for Fake.
Processed batch 1/10 for Fake...
Processed batch 2/10 for Fake...
Processed batch 3/10 for Fake...
Processed batch 4/10 for Fake...
Processed batch 5/10 for Fake...
Processed batch 6/10 for Fake...
Processed batch 7/10 for Fake...
Processed batch 8/10 for Fake...
Processed batch 9/10 for Fake...
Processed batch 10/10 for Fake...
Finished processing Fake dataset in 0.14 seconds.
Starting processing for Real dataset...
9 batches to process for Real.
Processed batch 1/9 for Real...
Processed batch 2/9 for Real...
Processed batch 3/9 for Real...
Processed batch 4/9 for Real...
Processed batch 5/9 for Real...
Processed batch 6/9 for Real...
Processed batch 7/9 for Real...
Processed batch 8/9 for Real...
Processed batch 9/9 for Real...
Finished processing Real dataset in 0.06 seconds.
Datasets saved as separate files:
All Images: 'all_images.pt', All Labels: 'all_labels.pt'
All Images Shape: torch.Size([374, 3, 256, 256]